In [1]:
import sys
import pandas as pd
import numpy as np
import urllib
import sqlalchemy
import string
import datetime

In [2]:
#Import shared functions
sys.path.append('..\..')
from IPM_Shared_Code_public.Python.utils import get_config

In [3]:
config = get_config('c:\Projects\config.ini')

driver = config['srv']['driver']
server = config['srv']['dev_server']
sladb = config['db']['sladb']

In [4]:
con_string = 'Driver={' + driver + '};Server=' + server +';Database=' + sladb + ';Trusted_Connection=Yes;'
params = urllib.parse.quote_plus(con_string)
engine = sqlalchemy.create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)

## Write all of the sql queries required for the tests

In [5]:
sql_sla_season = 'select * from sladb.dbo.tbl_sla_season'

In [6]:
sql_season_definition = 'select * from sladb.dbo.tbl_ref_sla_season_definition'

In [7]:
sql_season_date = 'select * from sladb.dbo.tbl_sla_season_date'

## Execute all of the sql queries and store the results

In [8]:
before_sla_season = pd.read_sql(con = engine, sql = sql_sla_season)

In [9]:
date_cols_season = [
    'effective_start', 'effective_end', 'created_date_utc', 'updated_date_utc',
    'effective_start_adj', 'effective_end_adj'
]

In [10]:
# pd.to_datetime(before_sla_season['effective_start'])

In [11]:
for i in date_cols_season:
    before_sla_season[i] = pd.to_datetime(before_sla_season[i])

In [12]:
before_sla_season.head()

,season_id,season_desc,year_round,effective,effective_end,effective_start,created_date_utc,updated_date_utc,effective_start_adj,effective_end_adj
0,1,"Year-round, not seasonal",True,False,2020-07-24,2014-01-01,2020-03-03 19:12:12.710,2020-07-28 20:16:56.850,2014-01-05,2020-07-25
1,2,"Beaches, etc.",False,False,2020-07-25,2019-07-01,2020-03-03 19:12:12.723,2020-07-29 12:39:31.760,2019-07-07,2020-07-25
2,3,"Ballfields, etc.",False,False,2020-07-25,2019-07-01,2020-03-03 19:12:12.723,2020-07-29 12:39:31.760,2019-07-07,2020-07-25
3,4,"NY Pause, COVID-19 Pandemic",True,False,2020-07-04,2020-03-22,2020-04-20 21:52:46.403,2020-07-01 15:20:36.193,2020-03-22,2020-07-04
4,6,New year round test season.,True,True,NaT,2020-07-24,2020-07-24 23:41:46.543,2020-07-28 18:31:55.867,2020-07-26,NaT


In [13]:
before_season_definition = pd.read_sql(con = engine, sql = sql_season_definition)

In [14]:
before_season_definition.head()

,season_date_ref_id,season_id,date_ref_fixed,month_name_desc,date_ref_day_number,day_name_desc,day_rank_id,date_type_id
0,1,1,True,January,1.0,None,None,1
1,2,1,True,December,31.0,None,None,2
2,3,2,True,May,1.0,None,None,1
3,4,2,True,October,1.0,None,None,2
4,5,3,True,March,1.0,None,None,1


In [15]:
before_season_date = pd.read_sql(con = engine, sql = sql_season_date)

In [16]:
date_cols_season_date = [
    'effective_start', 'effective_end', 'effective_start_adj',
    'effective_end_adj'
]

In [17]:
for i in date_cols_season_date:
    before_season_date[i] = pd.to_datetime(before_season_date[i])

In [18]:
before_season_date.head()

,season_date_id,season_id,effective_start,effective_end,date_category_id,effective_start_adj,effective_end_adj
0,1,1,2014-01-01,2020-07-24,1,2014-01-05,2020-07-25
1,2,2,2019-07-01,2019-10-01,1,2019-07-07,2019-10-05
2,4,2,2020-05-01,2020-07-25,1,2020-05-03,2020-07-25
3,6,3,2019-07-01,2019-10-31,1,2019-07-07,2019-11-02
4,8,3,2020-03-01,2020-07-25,1,2020-03-01,2020-07-25


In [19]:
timing = []

In [20]:
descriptions = []

In [21]:
results = []

## Before Tests

### First before test

In [22]:
timing.append('before')

In [23]:
descriptions.append('''Ensure that 'effective'==1 for every season that has\
'effective_end_date'==null''')

In [24]:
results.append(
    len(before_sla_season[(pd.isnull(before_sla_season['effective_end']))
                          & (before_sla_season['effective'] == False)]) == 0)

### Second before test

In [25]:
timing.append('before')

In [26]:
descriptions.append('''Test that 'effective_end_adj' is after today if \
'effective'==1 AND 'effective_end' != null in tbl_sla_season''')

In [27]:
before_sla_season['effective_end_adj'] = pd.to_datetime(
    before_sla_season['effective_end_adj'])

In [28]:
today = datetime.datetime.today()

In [29]:
temp_df = before_sla_season[(before_sla_season['effective'] == True)
                  & (~pd.isnull(before_sla_season['effective_end']))].copy()

In [30]:
results.append(len(temp_df[temp_df['effective_end_adj'] > today])==len(temp_df))

### Third before test

In [31]:
timing.append('before')

In [32]:
descriptions.append('''Test that every season in tbl_sla_season has exactly two\
entries in tbl_ref_sla_season_definition''')

In [33]:
entry_count = []
for i in before_sla_season.season_id.values:
    entry_count.append(
        len(before_season_definition[before_season_definition['season_id'] ==
                                     i]) == 2)

In [34]:
results.append(False not in entry_count)

### Fourth before test

In [35]:
timing.append('before')

In [36]:
descriptions.append('''Test that in tbl_ref_sla_season_definition if 'date_ref_fixed'==True,\
then columns 'month_name_desc' and 'date_ref_day_number' are not null ''')

In [37]:
temp_df = before_season_definition[before_season_definition['date_ref_fixed']
                                   == True]

In [38]:
results.append(
    len(temp_df) == len(temp_df[~(pd.isnull(temp_df['month_name_desc']))
                                &
                                (~pd.isnull(temp_df['date_ref_day_number']))]))

### Fifth before test

In [39]:
timing.append('before')

In [40]:
descriptions.append('''Test that in tbl_ref_sla_season_definition if 'date_ref_fixed'==False,\
then columns 'month_name_desc' and 'day_name_desc' and 'day_rank_id' are not null ''')

In [41]:
temp_df = before_season_definition[before_season_definition['date_ref_fixed']==False]

In [42]:
results.append(
    len(temp_df) == len(temp_df[~(pd.isnull(temp_df['month_name_desc']))
                                & (~pd.isnull(temp_df['day_name_desc']))
                                & (~pd.isnull(temp_df['day_rank_id']))]))

### Sixth before test

In [43]:
timing.append('before')

In [44]:
before_season_definition

,season_date_ref_id,season_id,date_ref_fixed,month_name_desc,date_ref_day_number,day_name_desc,day_rank_id,date_type_id
0,1,1,True,January,1.0,None,None,1
1,2,1,True,December,31.0,None,None,2
2,3,2,True,May,1.0,None,None,1
3,4,2,True,October,1.0,None,None,2
4,5,3,True,March,1.0,None,None,1
5,6,3,True,October,31.0,None,None,2
6,7,4,True,January,1.0,None,None,1
7,8,4,True,December,31.0,None,None,2
8,9,6,True,January,1.0,None,None,1
9,10,6,True,December,31.0,None,None,2


In [45]:
descriptions.append(
    '''For year-round seasons, test that 'date_ref_fixed'==1,\
    and start/end dates are Jan 1/Dec 31 ''')

In [46]:
temp_df = before_season_definition.merge(
    before_sla_season[before_sla_season['year_round'] == True],
    on='season_id')[[
        'season_id', 'year_round', 'date_ref_fixed', 'month_name_desc',
        'date_ref_day_number', 'date_type_id'
    ]]
temp_df

,season_id,year_round,date_ref_fixed,month_name_desc,date_ref_day_number,date_type_id
0,1,True,True,January,1.0,1
1,1,True,True,December,31.0,2
2,4,True,True,January,1.0,1
3,4,True,True,December,31.0,2
4,6,True,True,January,1.0,1
5,6,True,True,December,31.0,2
6,20,True,True,January,1.0,1
7,20,True,True,December,31.0,2
8,23,True,True,January,1.0,1
9,23,True,True,December,31.0,2


In [47]:
len(temp_df)==len(temp_df[((temp_df['date_type_id'] == 1)
         & (temp_df['month_name_desc'] == 'January')
         & (temp_df['date_ref_day_number'] == 1)) &
        (temp_df['date_ref_fixed'] == 1) |
        ((temp_df['date_type_id'] == 2)
         & (temp_df['month_name_desc'] == 'December')
        & (temp_df['date_ref_day_number'] == 31)) &
        (temp_df['date_ref_fixed'] == 1)])

True

In [48]:
results.append(
    len(temp_df) == len(temp_df[((temp_df['date_type_id'] == 1)
                                 & (temp_df['month_name_desc'] == 'January')
                                 & (temp_df['date_ref_day_number'] == 1))
                                & (temp_df['date_ref_fixed'] == 1)
                                | ((temp_df['date_type_id'] == 2)
                                   & (temp_df['month_name_desc'] == 'December')
                                   & (temp_df['date_ref_day_number'] == 31))
                                & (temp_df['date_ref_fixed'] == 1)]))

### Seventh before test

In [49]:
timing.append('before')

In [50]:
descriptions.append(
    '''For year-round seasons, test that 'effective_start' in tbl_sla_season_date\
    is equal to 'effective_start' in tbl_sla_season ''')

In [51]:
temp_df = before_season_date.merge(
    before_sla_season[before_sla_season['year_round'] == True],
    on='season_id')[[
        'season_id', 'year_round', 'effective_start_x', 'effective_start_y'
    ]]
temp_df

,season_id,year_round,effective_start_x,effective_start_y
0,1,True,2014-01-01,2014-01-01
1,4,True,2020-03-22,2020-03-22
2,6,True,2020-07-24,2020-07-24
3,20,True,2020-08-09,2020-08-09
4,23,True,2020-08-16,2020-08-16


In [52]:
results.append(
    len(temp_df) == len(temp_df[temp_df['effective_start_x'] ==
                                temp_df['effective_start_y']]))

### Eighth before test

In [53]:
timing.append('before')

In [54]:
descriptions.append(
    '''For periodic seasons, test that the earliest 'effective_start' for each season\
    in tbl_sla_season_date is equal to 'effective_start' in tbl_sla_season ''')

In [55]:
temp_df = before_season_date.merge(
    before_sla_season[before_sla_season['year_round'] == False],
    on='season_id')[[
        'season_id', 'year_round', 'effective_start_x', 'effective_start_y'
    ]].sort_values(by='effective_start_x').groupby('season_id').first()
temp_df

,year_round,effective_start_x,effective_start_y
season_id,,,
2,False,2019-07-01,2019-07-01
3,False,2019-07-01,2019-07-01
9,False,2020-07-26,2020-07-26
10,False,2020-07-26,2020-07-26
11,False,2020-08-09,2020-08-09
12,False,2020-08-07,2020-08-07
14,False,2020-08-13,2020-08-13
16,False,2020-08-15,2020-08-15
17,False,2020-08-16,2020-08-16


In [56]:
results.append(
    len(temp_df) == len(temp_df[temp_df['effective_start_x'] ==
                                temp_df['effective_start_y']]))

### Ninth before test

In [57]:
timing.append('before')

In [58]:
descriptions.append(
    '''For periodic seasons, test that the latest 'effective_end' for each season\
    in tbl_sla_season_date is equal to 'effective_end' in tbl_sla_season or today +1year(if null) ''')

In [59]:
temp_df = before_season_date.merge(
    before_sla_season[before_sla_season['year_round'] == False],
    on='season_id')[[
        'season_id', 'year_round', 'effective_start_x', 'effective_end_x', 'effective_end_y'
    ]].sort_values(by='effective_start_x').groupby('season_id').last()
temp_df

,year_round,effective_start_x,effective_end_x,effective_end_y
season_id,,,,
2,False,2020-05-01,2020-07-25,2020-07-25
3,False,2020-03-01,2020-07-25,2020-07-25
9,False,2020-07-26,2020-08-08,2020-08-08
10,False,2020-07-26,2020-08-08,2020-08-08
11,False,2021-03-01,2021-09-03,NaT
12,False,2020-08-07,2020-08-15,2020-08-15
14,False,2021-04-15,2021-09-03,NaT
16,False,2021-05-31,2021-09-03,NaT
17,False,2021-04-15,2021-09-03,NaT


In [60]:
two_conditions = []

In [61]:
two_conditions.append(len(temp_df[~pd.isnull(temp_df['effective_end_y'])]) == len(
    temp_df[(~pd.isnull(temp_df['effective_end_y']))
            & (temp_df['effective_end_x'] == temp_df['effective_end_y'])]))

In [62]:
two_conditions.append(
    len(temp_df[pd.isnull(temp_df['effective_end_y'])]) == len(temp_df[
        (pd.isnull(temp_df['effective_end_y']))
        & (temp_df['effective_end_x'] == datetime.datetime(
            today.year + 1, today.month, today.day))]))

In [63]:
results.append(False not in two_conditions)

## Make Updates

In [604]:
ninserts = 2

In [605]:
sla_id = [x for x in string.ascii_uppercase if x not in(before_distinct_sla_id.values)]

In [606]:
sla_inserts = [[x, 'Testing new SLA: '+ x, 1, 2] for x in sla_id]

In [607]:
sla_inserts_df = pd.DataFrame(sla_inserts[0:ninserts], 
                              columns = ['sla_id', 'sla_desc', 'sla_min_days', 'sla_max_days'])

In [608]:
sla_inserts_df

,sla_id,sla_desc,sla_min_days,sla_max_days
0,G,Testing new SLA: G,1,2
1,H,Testing new SLA: H,1,2


In [609]:
# sla_inserts_df.to_sql('tbl_ref_sla', con = engine, index = False, if_exists = 'append')

## Execute all of the sql queries AFTER UPDATES and store the results

In [8]:
after_sla_season = pd.read_sql(con = engine, sql = sql_sla_season)

In [9]:
date_cols_season = [
    'effective_start', 'effective_end', 'created_date_utc', 'updated_date_utc',
    'effective_start_adj', 'effective_end_adj'
]

In [11]:
for i in date_cols_season:
    after_sla_season[i] = pd.to_datetime(after_sla_season[i])

In [59]:
# before_sla_season.head()

In [13]:
after_season_definition = pd.read_sql(con = engine, sql = sql_season_definition)

In [60]:
# before_season_definition.head()

In [15]:
after_season_date = pd.read_sql(con = engine, sql = sql_season_date)

In [16]:
date_cols_season_date = [
    'effective_start', 'effective_end', 'effective_start_adj',
    'effective_end_adj'
]

In [17]:
for i in date_cols_season_date:
    after_season_date[i] = pd.to_datetime(after_season_date[i])

In [61]:
# after_season_date.head()

## List of tuples with final test results

In [85]:
list(zip(timing, descriptions, results))

[('before',
  "Ensure that 'effective'==1 for every season that has 'effective_end_date'==null",
  True),
 ('before',
  "Test that 'effective_end_adj' is after today if 'effective'==1 AND 'effective_end' != null in tbl_sla_season",
  True),
 ('before',
  'Test that every season in tbl_sla_season has exactly twoentries in tbl_ref_sla_season_definition',
  False),
 ('before',
  "Test that in tbl_ref_sla_season_definition if 'date_ref_fixed'==True,then columns 'month_name_desc' and 'date_ref_day_number' are not null ",
  True),
 ('before',
  "Test that in tbl_ref_sla_season_definition if 'date_ref_fixed'==False,then columns 'month_name_desc' and 'day_name_desc' and 'day_rank_id' are not null ",
  True),
 ('before',
  "For year-round seasons, test that 'effective_start' in tbl_sla_season_date    is equal to 'effective_start' in tbl_sla_season' ",
  True),
 ('before',
  "For periodic seasons, test that the earliest 'effective_start' for each season    in tbl_sla_season_date is equal to 'eff

In [64]:
pd.DataFrame(list(zip(timing, descriptions, results)), columns = {'before/after', 'test_desc', 'result'})             

,test_desc,before/after,result
0,before,Ensure that 'effective'==1 for every season th...,False
1,before,Test that 'effective_end_adj' is after today i...,True
2,before,Test that every season in tbl_sla_season has e...,True
3,before,Test that in tbl_ref_sla_season_definition if ...,True
4,before,Test that in tbl_ref_sla_season_definition if ...,True
5,before,"For year-round seasons, test that 'date_ref_fi...",True
6,before,"For year-round seasons, test that 'effective_s...",True
7,before,"For periodic seasons, test that the earliest '...",True
8,before,"For periodic seasons, test that the latest 'ef...",True
